In [7]:
import os
import pandas as pd
from datetime import datetime, date
from data_fetcher import YahooFinanceDataFetcher
from eda_utils import perform_eda
from analyzer import analyze_365_day_gain
from analyzer_max_drawdown import analyze_max_negative_gain
from backtester import backtest_weekly_investment
from plot_utils import plot_price_gain
from google_sheet_api import GoogleSheetsUploader

# === Setup ===
BASE_DIR = os.getcwd()
CONFIG_PATH = os.path.join(BASE_DIR, "config.yaml")
CREDENTIAL_PATH = os.path.join(BASE_DIR, "credential_google_sheets.json")
PLOTS_DIR = os.path.join(BASE_DIR, "plots")

def main():
    fetcher = YahooFinanceDataFetcher(CONFIG_PATH)
    symbol_data = fetcher.process_all_symbols()
    full_df = pd.concat(symbol_data.values(), ignore_index=True)

    # --- EDA Summary ---
    print("\n📊 EDA Summary:")
    print(perform_eda(full_df))

    final_summary, analyzed_data = [], {}

    for symbol, df in symbol_data.items():
        annotated_df, avg, upper, lower, latest_date, latest_price, pos_std, neg_std = analyze_365_day_gain(
            df, symbol, fetcher.std_multiplier
        )

        # --- Max Negative Gain Analysis ---
        df_maxloss, worst_drawdown, max_drawdown, _, _ = analyze_max_negative_gain(df, symbol)
        print(f"\n📉 {symbol} - Worst Future Gain : {worst_drawdown}% | Max Drawdown: {max_drawdown}%")

        if avg is None:
            continue

        upper_1std = round(pos_std, 2)
        lower_1std = round(-neg_std, 2)
        upper_1_97std = round(1.97 * pos_std, 2)
        lower_1_97std = round(-1.97 * neg_std, 2)

        plot_price_gain(annotated_df, symbol, avg, None, upper_1std, lower_1std, upper_1_97std, lower_1_97std, fetcher.std_multiplier)

        analyzed_data[symbol] = annotated_df

        # Compute max price (past 10 years)
        df["Datetime"] = pd.to_datetime(df["Datetime"], errors="coerce", utc=True)
        ten_years_ago = pd.Timestamp.now(tz='UTC') - pd.DateOffset(years=10)
        max_price = round(df[df["Datetime"] >= ten_years_ago]["Close"].max(), 2) if "Close" in df.columns else None

        # Simulate backtest using worst_drawdown instead of std
        portfolio_df, ar_invest, _, _ = backtest_weekly_investment(
            df,
            initial_balance=0,
            invest_per_week=200,
            tp_percent=1.0,
            leverage=1000,
            coeff=fetcher.coeff_map.get(symbol),
            std=abs(worst_drawdown),
            start_date="1900-01-01",
            end_date=str(date.today())
        )

        final_summary.append({
            "Symbol": symbol,
            "Date": latest_date,
            "Price": latest_price,
            "Max Price": max_price,
            "Worst Drawdown": abs(worst_drawdown),
            "Coefficient": fetcher.coeff_map.get(symbol),
            "Annual Return (Simulated)": ar_invest
        })

    # --- Final Summary ---
    final_df = pd.DataFrame(final_summary)
    print("\n✅ Final Summary:")
    print(final_df)

    # --- Upload to Google Sheets ---
    try:
        print("\n📤 Uploading to Google Sheets...")
        uploader = GoogleSheetsUploader(CREDENTIAL_PATH, "Financial Report - Indonesia")
        uploader.upload_dataframe(final_df, "Overview")
        print("✅ Upload successful!")
    except Exception as e:
        print(f"❌ Upload failed: {e}")

if __name__ == "__main__":
    main()

📈 Fetching AUDCAD (AUDCAD=X)...
📈 Fetching AUDCHF (AUDCHF=X)...
📈 Fetching AUDJPY (AUDJPY=X)...
📈 Fetching AUDNZD (AUDNZD=X)...
📈 Fetching AUDUSD (AUDUSD=X)...
📈 Fetching CADCHF (CADCHF=X)...
📈 Fetching CADJPY (CADJPY=X)...
📈 Fetching CHFJPY (CHFJPY=X)...
📈 Fetching EURAUD (EURAUD=X)...
📈 Fetching EURCAD (EURCAD=X)...
📈 Fetching EURCHF (EURCHF=X)...
📈 Fetching EURGBP (EURGBP=X)...
📈 Fetching EURJPY (EURJPY=X)...
📈 Fetching EURMXN (EURMXN=X)...
📈 Fetching EURNZD (EURNZD=X)...
📈 Fetching EURUSD (EURUSD=X)...
📈 Fetching EURZAR (EURZAR=X)...
📈 Fetching GBPAUD (GBPAUD=X)...
📈 Fetching GBPCAD (GBPCAD=X)...
📈 Fetching GBPCHF (GBPCHF=X)...
📈 Fetching GBPJPY (GBPJPY=X)...
📈 Fetching GBPMXN (GBPMXN=X)...
📈 Fetching GBPNZD (GBPNZD=X)...
📈 Fetching GBPUSD (GBPUSD=X)...
📈 Fetching GBPZAR (GBPZAR=X)...
📈 Fetching NZDCAD (NZDCAD=X)...
📈 Fetching NZDCHF (NZDCHF=X)...
📈 Fetching NZDJPY (NZDJPY=X)...
📈 Fetching NZDUSD (NZDUSD=X)...
📈 Fetching ZARJPY (ZARJPY=X)...
📈 Fetching USDCAD (CAD=X)...
📈 Fetching 


1 Failed download:
['STLA.MI']: YFPricesMissingError('possibly delisted; no price data found  (period=30y) (Yahoo error = "No data found, symbol may be delisted")')


📈 Fetching UCG.MIL (UCG.MI)...
📈 Fetching BBVA.BM (BBVA.MC)...
📈 Fetching CABK.BM (CABK.MC)...
📈 Fetching ITX.BM (ITX.MC)...
📈 Fetching SAN.BM (SAN.MC)...
📈 Fetching ADS.XE (ADS.DE)...
📈 Fetching ALV.XE (ALV.DE)...
📈 Fetching BAYN.FWB (BAYN.DE)...
📈 Fetching BEI.XE (BEI.DE)...
📈 Fetching BMW.XE (BMW.DE)...
📈 Fetching DTE.FWB (DTE.DE)...
📈 Fetching DTG.FWB (DTG.DE)...
📈 Fetching MBG.XE (MBG.DE)...
📈 Fetching MRK.XE (MRK.DE)...
📈 Fetching SIE.XE (SIE.DE)...
📈 Fetching VNA.XE (VNA.DE)...
📈 Fetching VOW3.XE (VOW3.DE)...
📈 Fetching ASML.EAS (ASML.AS)...
📈 Fetching HEIA.EAS (HEIA.AS)...
📈 Fetching CPTL.SGX (9CI.SI)...
📈 Fetching JAR.SGX (C07.SI)...
📈 Fetching DAII.TSE (8750.T)...
📈 Fetching DKI.TSE (6367.T)...
📈 Fetching HIT.TSE (6501.T)...
📈 Fetching KEE.TSE (6861.T)...
📈 Fetching MUR.TSE (6981.T)...
📈 Fetching NID.TSE (9594.T)...



1 Failed download:
['9594.T']: YFPricesMissingError('possibly delisted; no price data found  (period=30y) (Yahoo error = "No data found, symbol may be delisted")')


📈 Fetching OL.TSE (4661.T)...
📈 Fetching SVN.TSE (3382.T)...
📈 Fetching TKY.TSE (8035.T)...
📈 Fetching TM.TSE (7203.T)...
📈 Fetching TMH.TSE (8766.T)...
📈 Fetching NESTE.OMXH (NESTE.HE)...
📈 Fetching NOKIA.OMXH (NOKIA.HE)...
📈 Fetching TCEHY.OTC (TCEHY)...
📈 Fetching AAVEUSD (AAVE-USD)...
📈 Fetching ADAUSD (ADA-USD)...
📈 Fetching ALGOUSD (ALGO-USD)...
📈 Fetching APTUSD (APT21794-USD)...
📈 Fetching ARBUSD (ARB11841-USD)...
📈 Fetching ATOMUSD (ATOM-USD)...
📈 Fetching AVAXUSD (AVAX-USD)...
📈 Fetching AXSUSD (AXS-USD)...
📈 Fetching BCHUSD (BCH-USD)...
📈 Fetching BNBUSD (BNB-USD)...
📈 Fetching BTCUSD (BTC-USD)...
📈 Fetching DASHUSD (DASH-USD)...
📈 Fetching DOGEUSD (DOGE-USD)...
📈 Fetching DOTUSD (DOT-USD)...
📈 Fetching ETCUSD (ETC-USD)...
📈 Fetching ETHUSD (ETH-USD)...
📈 Fetching FETUSD (FET-USD)...
📈 Fetching FILUSD (FIL-USD)...
📈 Fetching GRTUSD (GRT-USD)...
📈 Fetching ICPUSD (ICP-USD)...
📈 Fetching IOTAUSD (IOTA-USD)...
📈 Fetching LINKUSD (LINK-USD)...
📈 Fetching LRCUSD (LRC-USD)...
📈 Fe